In [1]:
import numpy as np
import os
from pathlib import Path
import shutil
import json
import os.path as op

basedir = Path('/data/ds-siemenspilots24/') #sourcedata/mri'

subject = 'alina'

In [2]:
orders = [[0, 2, 4], [4, 2, 0], [2, 0, 4]]

In [3]:
template_json = {"MagneticFieldStrength": 3,
                "ParallelReductionFactorInPlane": 1,
                "RepetitionTime": 0.0,
                "TaskName": "Numerosity estimation",
                "PhaseEncodingDirection": "i",
                'TotalReadoutTime': 0.04,}

In [4]:
from nilearn import image, plotting

In [ ]:
import json

def get_lr_encoding(session, run):
    mb = orders[session-1][(run-1) % 3]
    repetition = (run-1) // 3 + 1

    assert mb in [0, 2, 4]

    if repetition == 1:
        if mb in [0, 4]:
            return 'LR'
        else:
            return 'RL'
    else:
        if mb in [0, 4]:
            return 'RL'
        else:
            return 'LR'
    

for session in [1,2,3]:

    target_dir = basedir / f'sub-{subject}' / f'ses-{session}'
    source_dir = basedir / 'sourcedata' / 'mri' / f'sub-{subject}' / f'01_{session:02d}'

    os.makedirs(target_dir, exist_ok=True)

    order = orders[session-1]

    for sd in ['anat', 'fmap', 'func']:
        os.makedirs(target_dir / sd, exist_ok=True)

    # Find anatomical data:
    anat_files = list(source_dir.glob('t1_mprage*.nii.gz'))

    for i, anat_fn in enumerate(anat_files):
        if len(anat_files) == 1:
            target_fn = target_dir / 'anat' / f'sub-{subject}_ses-{session}_T1w.nii.gz'
        else:
            target_fn = target_dir / 'anat' / f'sub-{subject}_ses-{session}_run-{i+1}_T1w.nii.gz'

        shutil.copy(anat_fn, target_fn)

    # Find functional data:

    for i, mb in enumerate(order):

        for repetition in [1, 2]:

            # lr = ['LR', 'RL'][run % 2]
            run = i+1 + 3*(repetition-1)

            lr = get_lr_encoding(session, run)

            source_fn = source_dir / f'fmri_g0_sms{mb}_f8p8_r{repetition}.nii.gz'
            target_fn = target_dir / 'func' / f'sub-{subject}_ses-{session}_task-numestimate_acq-mb{mb}_dir-{lr}_run-{run:02d}_bold.nii.gz'


            shutil.copy(source_fn, target_fn)

            json_dict = template_json.copy()
            json_dict['RepetitionTime'] = float(image.load_img(source_fn).header.get_zooms()[3])
            json_dict['PhaseEncodingDirection'] = 'i-' if lr == 'RL' else 'i'
            json_dict['MultibandAccelerationFactor'] = mb
            json_dict['EchoTime'] = 0.03
            json_dict['TotalReadoutTime'] = 0.05282
            json_dict['EffectiveEchoSpacing'] = 0.00074
            json_dict['EPIFactor'] = 72
            json_dict['BandwidthPerPixelPhaseEncode'] = 2272

            with open(target_fn.with_suffix('').with_suffix('.json'), 'w') as f:
                json.dump(json_dict, f)

            # If mulitband 1, write it down as sbref

            if mb == 0:
                mbref = image.load_img(source_fn).slicer[:, :, :, 0]

                print(f"**** Preparing sbref for run ***", run)

                target_runs = list(range(1, 7))
                target_runs.pop(run-1)

                for target_run in target_runs:
                    print(f"Checking target run {target_run}")
                    
                    mb_target = order[(target_run-1) % 3]
                    lr_target = get_lr_encoding(session, target_run)

                    print('mb_target', mb_target)
                    print('lr_target', lr_target, 'my lr:', lr)

                    if (mb_target !=0) & (lr_target == lr):
                        print(f"Found target run {target_run}")
                        target_fn = target_dir / 'func' / f'sub-{subject}_ses-{session}_task-numestimate_acq-mb{mb_target}_dir-{lr}_run-{target_run:02d}_sbref.nii.gz'
                        mbref.to_filename(target_fn)

                        json_fn = target_fn.with_suffix('').with_suffix('.json')
                        with open(json_fn, 'w') as f:
                            json.dump(json_dict, f)


    for i, mb in enumerate(order):

        for repetition in [1, 2]:

            run = i+1 + 3*(repetition-1)

            other_repetition = 1 if repetition == 2 else 2
            source_fn = source_dir / f'fmri_g0_sms{mb}_f8p8_r{repetition}.nii.gz'

            lr = get_lr_encoding(session, run)

            target_run = i+1 + 3*(other_repetition-1)
            target_lr = get_lr_encoding(session, target_run)

            assert(lr != target_lr)

            if repetition == 1:
                # Get last 5 slices
                topup = image.load_img(source_fn).slicer[:, :, :, -5:]
                bold_fn = Path(f'ses-{session}') / 'func' / f'sub-{subject}_ses-{session}_task-numestimate_acq-mb{mb}_dir-{target_lr}_run-{i+1+3:02d}_bold.nii.gz'
            else:
                # Get first 5 slices
                topup = image.load_img(source_fn).slicer[:, :, :, :5]
                bold_fn = Path(f'ses-{session}') / 'func' / f'sub-{subject}_ses-{session}_task-numestimate_acq-mb{mb}_dir-{target_lr}_run-{i+1:02d}_bold.nii.gz'

            assert(basedir / f'sub={subject}' / bold_fn)

            #sub-alina_ses-3_acq-mb0_dir-RL_epi.nii.gz
            target_fn = target_dir / 'fmap' / f'sub-{subject}_ses-{session}_acq-mb{mb}_dir-{lr}_epi.nii.gz'
            topup.to_filename(target_fn)

            target_json_fn = target_fn.with_suffix('').with_suffix('.json')

            json_dict['IntendedFor'] = [str(bold_fn)]
            json_dict['PhaseEncodingDirection'] = 'i-' if lr == 'RL' else 'i'

            with open(target_json_fn, 'w') as f:
                json.dump(json_dict, f)

**** Preparing sbref for run *** 1
Checking target run 2
mb_target 2
lr_target RL my lr: LR
Checking target run 3
mb_target 4
lr_target LR my lr: LR
Found target run 3
Checking target run 4
mb_target 0
lr_target RL my lr: LR
Checking target run 5
mb_target 2
lr_target LR my lr: LR
Found target run 5
Checking target run 6
mb_target 4
lr_target RL my lr: LR
**** Preparing sbref for run *** 4
Checking target run 1
mb_target 0
lr_target LR my lr: RL
Checking target run 2
mb_target 2
lr_target RL my lr: RL
Found target run 2
Checking target run 3
mb_target 4
lr_target LR my lr: RL
Checking target run 5
mb_target 2
lr_target LR my lr: RL
Checking target run 6
mb_target 4
lr_target RL my lr: RL
Found target run 6
**** Preparing sbref for run *** 3
Checking target run 1
mb_target 4
lr_target LR my lr: LR
Found target run 1
Checking target run 2
mb_target 2
lr_target RL my lr: LR
Checking target run 4
mb_target 4
lr_target RL my lr: LR
Checking target run 5
mb_target 2
lr_target LR my lr: LR
Fo

# With fieldmap 

In [7]:

basedir = Path('/data/ds-siemenspilotsfmap/') #sourcedata/mri'

subject = 'alina'

In [12]:
import json

def get_lr_encoding(session, run):
    mb = orders[session-1][(run-1) % 3]
    repetition = (run-1) // 3 + 1

    assert mb in [0, 2, 4]

    if repetition == 1:
        if mb in [0, 4]:
            return 'LR'
        else:
            return 'RL'
    else:
        if mb in [0, 4]:
            return 'RL'
        else:
            return 'LR'
    

for session in [1,2,3]:

    target_dir = basedir / f'sub-{subject}' / f'ses-{session}'
    source_dir = basedir / 'sourcedata' / 'mri' / f'sub-{subject}' / f'01_{session:02d}'

    os.makedirs(target_dir, exist_ok=True)

    order = orders[session-1]

    for sd in ['anat', 'fmap', 'func']:
        os.makedirs(target_dir / sd, exist_ok=True)

    # Find anatomical data:
    anat_files = list(source_dir.glob('t1_mprage*.nii.gz'))

    for i, anat_fn in enumerate(anat_files):
        if len(anat_files) == 1:
            target_fn = target_dir / 'anat' / f'sub-{subject}_ses-{session}_T1w.nii.gz'
        else:
            target_fn = target_dir / 'anat' / f'sub-{subject}_ses-{session}_run-{i+1}_T1w.nii.gz'

        shutil.copy(anat_fn, target_fn)

    # Find functional data:

    bold_fns = []

    for i, mb in enumerate(order):

        for repetition in [1, 2]:

            # lr = ['LR', 'RL'][run % 2]
            run = i+1 + 3*(repetition-1)

            lr = get_lr_encoding(session, run)

            source_fn = source_dir / f'fmri_g0_sms{mb}_f8p8_r{repetition}.nii.gz'
            target_fn = target_dir / 'func' / f'sub-{subject}_ses-{session}_task-numestimate_acq-mb{mb}_dir-{lr}_run-{run:02d}_bold.nii.gz'

            bold_fns.append(Path(f'ses-{session}') / 'func' / f'sub-{subject}_ses-{session}_task-numestimate_acq-mb{mb}_dir-{lr}_run-{run:02d}_bold.nii.gz')

            shutil.copy(source_fn, target_fn)

            json_dict = template_json.copy()
            json_dict['RepetitionTime'] = float(image.load_img(source_fn).header.get_zooms()[3])
            json_dict['PhaseEncodingDirection'] = 'i-' if lr == 'RL' else 'i'
            json_dict['MultibandAccelerationFactor'] = mb
            json_dict['EchoTime'] = 0.03
            json_dict['TotalReadoutTime'] = 0.05282
            json_dict['EffectiveEchoSpacing'] = 0.00074
            json_dict['EPIFactor'] = 72
            json_dict['BandwidthPerPixelPhaseEncode'] = 2272

            with open(target_fn.with_suffix('').with_suffix('.json'), 'w') as f:
                json.dump(json_dict, f)

            # If mulitband 1, write it down as sbref

            if mb == 0:
                mbref = image.load_img(source_fn).slicer[:, :, :, 0]

                print(f"**** Preparing sbref for run ***", run)

                target_runs = list(range(1, 7))
                target_runs.pop(run-1)

                for target_run in target_runs:
                    
                    mb_target = order[(target_run-1) % 3]
                    lr_target = get_lr_encoding(session, target_run)

                    if (mb_target !=0) & (lr_target == lr):
                        print(f"Found target run {target_run}")
                        target_fn = target_dir / 'func' / f'sub-{subject}_ses-{session}_task-numestimate_acq-mb{mb_target}_dir-{lr}_run-{target_run:02d}_sbref.nii.gz'
                        mbref.to_filename(target_fn)

                        json_fn = target_fn.with_suffix('').with_suffix('.json')
                        with open(json_fn, 'w') as f:
                            json.dump(json_dict, f)


    fieldmap_echo1 = source_dir / 'gre_field-mapping_nd_e1.nii.gz'
    fieldmap_echo2 = source_dir / 'gre_field-mapping_nd_e2.nii.gz'
    fieldmap_phase = source_dir / 'gre_field-mapping_nd_e2_ph.nii.gz'




    os.makedirs(target_dir / 'fmap', exist_ok=True)

    shutil.copy(fieldmap_echo1, target_dir / 'fmap' / f'sub-{subject}_ses-{session}_magnitude1.nii.gz')
    shutil.copy(fieldmap_echo2, target_dir / 'fmap' / f'sub-{subject}_ses-{session}_magnitude2.nii.gz')
    shutil.copy(fieldmap_phase, target_dir / 'fmap' / f'sub-{subject}_ses-{session}_phasediff.nii.gz')

    fmap_json = {'EchoTime1': 0.00492,
                'EchoTime2': 0.00738,
                'FieldOfViewPhaseEncoding': 0.818,
                'BandwidthPerPixelPhaseEncode': 2272,
                'IntendedFor':[str(e) for e in bold_fns]}

    with open(target_dir / 'fmap' / f'sub-{subject}_ses-{session}_phasediff.json', 'w') as f:
        json.dump(fmap_json, f)

**** Preparing sbref for run *** 1
Found target run 3
Found target run 5
**** Preparing sbref for run *** 4
Found target run 2
Found target run 6
**** Preparing sbref for run *** 3
Found target run 1
Found target run 5
**** Preparing sbref for run *** 6
Found target run 2
Found target run 4
**** Preparing sbref for run *** 2
Found target run 3
Found target run 4
**** Preparing sbref for run *** 5
Found target run 1
Found target run 6
